In [1]:
import pandas as pd
import monai
import os
import numpy as np
import dcm2niix
import pydicom
import shutil
from pydicom.errors import InvalidDicomError
from tqdm import tqdm
import nibabel as nib
import SimpleITK as sitk
from Preprocessing import NIFTI_preparation_functions as nifp
from Preprocessing import DICOM_preparation_functions as dicp
import yaml
from monai.transforms import  SaveImage
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import pprint

In [2]:
with open('./config_Small.yaml', 'r') as ymlfile:
    cfg = yaml.safe_load(ymlfile)

x_image_size = cfg['data_preparation']['image_size_x']
y_image_size = cfg['data_preparation']['image_size_y']
z_image_size = cfg['data_preparation']['image_size_z']
DICOM_FOLDER = cfg['preprocessing']['root_dicom_folder']

In [3]:
root_dataFolder = os.path.split(DICOM_FOLDER)[0]

In [4]:
Dataset_direc = None
NIFTI_direc = os.path.join(root_dataFolder,'NIFTI')
NIFIT_slices_direc = os.path.join(root_dataFolder,'NIFTI_SLICES')
if not os.path.exists(NIFTI_direc):
    os.mkdir(NIFTI_direc)
if not os.path.exists(NIFIT_slices_direc):
    os.mkdir(NIFIT_slices_direc)

In [5]:
if Dataset_direc:
    for root, dirs, files in os.walk(Dataset_direc):
        for i_file in files:
            if '.nii.gz' in i_file:
                filepath = os.path.join(root,i_file)
                shutil.move(filepath,os.path.join(NIFTI_direc,i_file))
            if '.dcm' in i_file:
                filepath = os.path.join(root,i_file)
                shutil.move(filepath,os.path.join(DICOM_FOLDER,i_file))

In [6]:
dicp.sort_DICOM_to_structured_folders(root_dir=DICOM_FOLDER)

'../data_small/DICOM_STRUCTURED'

In [7]:
nifp.convert_DICOM_to_NIFTI(root_dir='../data_small/DICOM_STRUCTURED')

Chris Rorden's dcm2niiX version v1.0.20220505  GCC11.4.0 x86-64 (64-bit Linux)
n.b. Setting directory search depth of zero invokes internal gz (network mode)
Found 22 DICOM file(s)
Maximizing 16-bit range: raw 0..939 is34
Convert 22 DICOM as ../data_small/TEMP/20061201_MR_2.16.840.1.113669.632.20.1211.10000357775_1.3.46.670589.11.0.0.11.4.2.0.8743.5.5396.2006120114262848496 (512x512x22x1)
Conversion required 0.506444 seconds (0.506425 for core code).
Chris Rorden's dcm2niiX version v1.0.20220505  GCC11.4.0 x86-64 (64-bit Linux)
n.b. Setting directory search depth of zero invokes internal gz (network mode)
Found 22 DICOM file(s)
Maximizing 16-bit range: raw 0..1884 is16
Convert 22 DICOM as ../data_small/TEMP/20061201_MR_2.16.840.1.113669.632.20.1211.10000357775_1.3.46.670589.11.0.0.11.4.2.0.8743.5.5396.2006120114333440565 (512x512x22x1)
Conversion required 0.953499 seconds (0.953455 for core code).
Chris Rorden's dcm2niiX version v1.0.20220505  GCC11.4.0 x86-64 (64-bit Linux)
n.b. Setti

'../data_small/NIFTI'

In [8]:
images_4D_file = nifp.extract_4D_images(NIFTI_direc)

0it [00:00, ?it/s]
100%|██████████| 4/4 [00:00<00:00, 25.10it/s]


In [9]:
filepaths = []
filenames = []
for root, dirs, files in os.walk(NIFTI_direc):
    for i_file in files:
        if '.nii.gz' in i_file:
            filepaths.append(os.path.join(root,i_file))
            filenames.append(i_file.split('.nii.gz')[0])
print(len(filepaths))
dataset = [{'image':filepath,'label':filename} for filepath, filename in zip(filepaths,filenames)]

4


In [10]:
train_dataset, test_dataset = train_test_split(dataset,random_state=42,shuffle=True,test_size=4)

ValueError: test_size=4 should be either positive and smaller than the number of samples 4 or a float in the (0, 1) range

In [11]:
test_dataset = dataset

In [12]:
class SaveIndividualSlices(monai.transforms.MapTransform):
    def __init__(self, keys, targetDir, split):
        self.keys = keys
        self.targetDir = targetDir
        self.split = split

    def __call__(self, data):
        os.makedirs(self.targetDir,exist_ok=True)
        depth = data[self.keys[0]].shape[-1]
        for i in range(depth):
            saver = SaveImage(
                output_dir=os.path.join(self.targetDir,self.split),
                output_postfix=f'_s{i}',
                output_ext='.nii.gz',
                resample=False,
                squeeze_end_dims=True,
                data_root_dir=NIFTI_direc,
                separate_folder=False,
                print_log=False,
                savepath_in_metadict=True,
            )
            slice = data[self.keys[0]][:,:,:,i]
            slice = np.array(slice)
            slice = slice.reshape((1,x_image_size,y_image_size,1))
            metadata = dict(data[f'{self.keys[0]}_meta_dict'])
            saver(img=slice,meta_data=metadata)
            # filename = str(data[self.keys[1]])
            # savepath = os.path.join(self.targetDir,f'{filename}__s{i}.nii.gz')
            # affine = metadata['affine']
            # nib.save(img=nib.Nifti1Pair(slice,affine=affine), filename=savepath)
        return data

In [13]:
trainTransform = monai.transforms.Compose(
    [
        monai.transforms.LoadImaged(keys=['image'], image_only=False, ensure_channel_first=False),
        monai.transforms.EnsureChannelFirstd(keys=['image']),
        monai.transforms.Orientationd(keys=['image'],axcodes='LAS'),
        monai.transforms.Resized(keys=['image'], spatial_size=[x_image_size, y_image_size, z_image_size]),
        monai.transforms.ScaleIntensityd(keys=['image']),
        SaveIndividualSlices(keys=['image', 'label'], targetDir=NIFIT_slices_direc,split='train')
    ]
)
testTransform = monai.transforms.Compose(
    [
        monai.transforms.LoadImaged(keys=['image'], image_only=False, ensure_channel_first=False),
        monai.transforms.EnsureChannelFirstd(keys=['image']),
        monai.transforms.Orientationd(keys=['image'],axcodes='LAS'),
        monai.transforms.Resized(keys=['image'], spatial_size=[x_image_size, y_image_size, z_image_size]),
        monai.transforms.ScaleIntensityd(keys=['image']),
        SaveIndividualSlices(keys=['image', 'label'], targetDir=NIFIT_slices_direc,split='test')
    ]
)

In [0]:
ds_train = monai.data.Dataset(train_dataset, trainTransform)
dl_train = monai.data.DataLoader(ds_train, batch_size=1, num_workers=1)

In [16]:
ds_test = monai.data.Dataset(test_dataset, testTransform)
dl_test = monai.data.DataLoader(ds_test, batch_size=1, num_workers=1)

In [ ]:
counter = -1
try:
    for i in tqdm(dl_train):
        counter+=1
        continue
except:
    print(i['label'])

In [17]:
counter = -1
try:
    for i in tqdm(dl_test):
        counter+=1
        continue
except:
    print(i['label'])

100%|██████████| 4/4 [00:01<00:00,  2.23it/s]


In [18]:
nifp.create_label_file(root_dir=os.path.join(NIFIT_slices_direc,'train'),base_dir=root_dataFolder,images_4D_file=images_4D_file,name='trainLabelsSmall.txt')
nifp.create_label_file(root_dir=os.path.join(NIFIT_slices_direc,'test'),base_dir=root_dataFolder,images_4D_file=images_4D_file,name='testLabelsSmall.txt')

genfromtxt: Empty input file: "../data_small/DATA/Scans_4D.txt"


'../data_small/DATA/testLabelsSmall.txt'